cos-sync

syncs two S3 compliant COS buckets (unidirectional)

In [2]:
!pip install kfp kfp-tekton

  Using cached kfp-2.0.1.tar.gz (372 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached kfp_tekton-1.7.1-py3-none-any.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 2.3 MB/s eta 0:00:00a 0:00:01
  Using cached docstring_parser-0.15-py3-none-any.whl (36 kB)
  Using cached google_api_core-2.11.1-py3-none-any.whl (120 kB)
  Using cached google_auth-2.21.0-py2.py3-none-any.whl (182 kB)
  Using cached google_cloud_storage-2.10.0-py2.py3-none-any.whl (114 kB)
  Using cached kfp_pipeline_spec-0.2.2-py3-none-any.whl (20 kB)
  Using cached kfp-server-api-2.0.0.tar.gz (63 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached kubernetes-26.1.0-py2.py3-none-any.whl (1.4 MB)
  Using cached protobuf-3.20.3-py2.py3-none-any.whl (162 kB)
  Using cached PyYAML-6.0-cp311-cp311-manylinux_2

In [4]:
import kfp
from kfp.components import create_component_from_func
from kfp_tekton.compiler import TektonCompiler
import kfp.dsl as dsl
import kfp.dsl as dsl
import kfp
from kfp_tekton.compiler import TektonCompiler
from kfp.compiler import Compiler
import kfp.components as comp


In [5]:
 
util_cos = comp.load_component_from_file('/home/romeokienzler/gitco/component-library/component-library/util/util-cos.yaml')

In [9]:


# Define your pipeline function
@dsl.pipeline(
    name="cos-sync",
    description="syncs two S3 compliant COS buckets (unidirectional)"
)
def agb_inference_preprocess(
        source_cos_path: str,
        source_access_key_id: str,
        source_secret_access_key: str,
        source_endpoint: str,
        source_bucket_name: str,
        target_local_path: str,
        target_access_key_id: str,
        target_secret_access_key: str,
        target_endpoint: str,
        target_bucket_name: str,
        buffer_dir: str = '/data/',
        pvc_mount_path: str = '/data/',
        pvc_name: str = 'kfp-data-pvc-z'
,
):
  
    step1 = util_cos(
            access_key_id=source_access_key_id,
            secret_access_key=source_secret_access_key,
            endpoint=source_endpoint,
            bucket_name=source_bucket_name,
            path=source_cos_path,
            recursive='True',
            operation='sync_to_local',
            target=buffer_dir,
    )

    step2 = util_cos(
            access_key_id=target_access_key_id,
            secret_access_key=target_secret_access_key,
            endpoint=target_endpoint,
            bucket_name=target_bucket_name,
            source=str(buffer_dir)+str(target_local_path),
            recursive='True',
            operation='sync_to_cos',
    )

    step2.after(step1)


    pipeline_volume = dsl.PipelineVolume(pvc=pvc_name)


    step1.add_pvolumes({pvc_mount_path: pipeline_volume})
    step2.add_pvolumes({pvc_mount_path: pipeline_volume})

TektonCompiler().compile(agb_inference_preprocess, 'cos-sync.tekton.yaml')
Compiler().compile(agb_inference_preprocess, 'cos-sync.argo.yaml')

TypeError: unsupported operand type(s) for +: 'PipelineParam' and 'PipelineParam'